# Use the VLM endpoint

In [ ]:
import base64
import cv2
import numpy as np
import os
import pyheif

from typing import Optional

from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from openai import AzureOpenAI, ChatCompletion
from IPython.display import display, Markdown


# Function to encode an image
def encode_image(image_path: str):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

# Function to encode and resize an image
def encode_and_resize(image_path: str, max_size: int = 640):
    if image_path.endswith(".HEIC"):
        img = read_heic_to_numpy(image_path)
    else:
        img = cv2.imread(image_path)
    scale_factor = max_size / max(img.shape)
    img = cv2.resize(img, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_LINEAR)
    _, im_arr = cv2.imencode('.jpg', img)  # im_arr: image in Numpy one-dim array format.
    im_bytes = im_arr.tobytes()
    return base64.b64encode(im_bytes).decode("utf-8")

# Function to read .HEIC image format
def read_heic_to_numpy(file_path: str):
    heif_file = pyheif.read(file_path)
    data = heif_file.data
    if heif_file.mode == "RGB":
        numpy_array = np.frombuffer(data, dtype=np.uint8).reshape(
            heif_file.size[1], heif_file.size[0], 3)
    elif heif_file.mode == "RGBA":
        numpy_array = np.frombuffer(data, dtype=np.uint8).reshape(
            heif_file.size[1], heif_file.size[0], 4)
    else:
        raise ValueError("Unsupported HEIC color mode")
    return numpy_array


endpoint = "https://oai-aip-cv-ont-sdc.openai.azure.com/"
model_name = "gpt-4o"
deployment = "gpt-4o"
token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")
api_version = "2024-12-01-preview"

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    azure_ad_token_provider=token_provider,
)

In [ ]:
def prompt(system_prompt: str, user_prompt: str, image_path: Optional[str] = None, detail: str = "low") -> ChatCompletion:
    system_message = {
        "role": "system",
        "content": system_prompt
    }

    user_content = [
        { "type": "text", "text": user_prompt }
    ]
    if image_path is not None:
        base64_image = encode_and_resize(image_path)
        user_content.append(
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{base64_image}",
                    "detail": detail, # reduces token usage
                }
            }
        )

    user_message = {
        "role": "user",
        "content": user_content
    }

    messages = [
        system_message,
        user_message,
    ]

    response = client.chat.completions.create(
        messages=messages,
        max_tokens=4096,
        temperature=0.0,
        model=deployment
    )

    return response

## Ask question with a photo as context

In [ ]:
SYSTEM_PROMPT = """
Je bent een AI-assistent die foto's van de openbare ruimte beoordeelt op basis van de CROW-systematiek gericht op voetgangersveiligheid, met een focus op struikelgevaar. Analyseer de foto op de volgende aspecten:

1. **Oneffenheden in het oppervlak**: Identificeer hoogteverschillen, losse of middende elementen in de verharding, of gaten.
2. **Continuïteit van het loopvlak**: Controleer of het loopvlak onderbroken wordt en of dit voetgangers dwingt om obstakels te vermijden, uit balans te raken, of ongebruikelijke stappen te zetten.
3. **Materiaal en conditie**: Beoordeel materiaalbeschadiging zoals scheuren, verzakkingen, of gladde oppervlakken die struikelrisico’s kunnen verhogen.
4. **Omgevingsfactoren**: Houd rekening met omgevingskenmerken zoals slechte verlichting, scherpe bochten of verwaarloosde plekken die risico's kunnen verergeren.  

Geef een prioriteitsscore:  
- **Laag** risico: Klein of verwaarloosbaar struikelgevaar, interventie kan wachten. Dit is bijvoorbeeld het geval wanneer de oneffenheid kleiner is dan 1cm.
- **Middel** risico: Aandacht vereist, plan remedie op middellange termijn. Dit is bijvoorbeeld het geval wanneer de oneffenheid tussen de 1cm en 3cm is.
- **Hoog** risico: Directe actie noodzakelijk vanwege significante kans op letsel. Dit is bijvoorbeeld het geval wanneer de oneffenheid groter is dan 3cm.

Formatteer je analyse als een inspectierapport.

Inspectierapport
---
**Omschrijving van het probleem:** [Beknopte omschrijving van het belangrijkste probleem, bijvoorbeeld "Losliggende tegel", "Boomwortelschade", "Gat"].
**Omgevingsfactoren:** [Bijv. Slecht zicht, aanwezige obstakels, weersinvloed]
**Hoogteverschil:** [Schatting van het hoogteverschil in cm]
**Beoordeling van risico:** {Laag, Middel, Hoog}
**Toelichting:** [Geef een beknopte technische verklaring voor de risico score, gebruikmakend van je analyse. Refereer specifieke elementen in de foto. Voorbeeld: "De foto toont een losliggende stoeptegel in het midden van de looproute. De oneffenheid is groter dan 3cm en valt dus onder hoog risico."] 
---

Als er geen duidelijk risico zichtbaar is, geef dan "Geen significant risico" als **Omschrijving van het probleem** en geef als prioriteit "Laag".
"""

USER_PROMPT = """
Genereer een inspectierapport voor deze foto.
"""

In [ ]:

# images_folder = "../datasets/experiments/onderhoud/Collection/"  # See other notebook on how to download an image from blob store
# image_name = "IMG_1182.HEIC"

images_folder = "../datasets/experiments/onderhoud/2025_Centrum/Foto's klein onderhoud/"
image_name = "59ef2cf7-b13c-4f9e-8878-9752e015cb56.jpeg"

response = prompt(SYSTEM_PROMPT, USER_PROMPT, image_path=os.path.join(images_folder, image_name), detail="high")

text = response.choices[0].message.content

display(Markdown(text.replace("/n", "<br>")))

## GPT prompt engineering

In [ ]:
SYSTEM_PROMPT = """
Je bent een AI hulpmiddel dat de Gemeente Amsterdam ondersteunt bij het analyseren van foto's in de openbare ruimte. 
Je specifieke doel is om struikelgevaar voor voetgangers te beoordelen op basis van de CROW-systematiek voor wegbeheer.
"""

USER_PROMPT = """
Geef een system prompt die geschikt is om foto's te beoordelen op basis van de CROW methodiek, specifiek toegespitst op struikelgevaar. 
Gebruik CROW elementen in de prompt en zorg dat het antwoord een prioriteit score bevat van {laag, middel, hoog} gebaseerd op het risico.
Geef ook een indicatie van de termijn waarop het probleem verholpen moeten worden. Oneffenheden van meer dan 3cm zijn urgent.
De output moet geformateerd worden als inspectierapport.
"""

response = prompt(SYSTEM_PROMPT, USER_PROMPT)

print(response.choices[0].message.content)

## Extract description and risk from response and generate a dataframe

In [ ]:
import pandas as pd

val_df = pd.read_csv("../datasets/experiments/onderhoud/2025_Centrum/validatieset.csv", sep=";")
images_folder = "../datasets/experiments/onderhoud/2025_Centrum/validatie/"


data = {
    "foto": [],
    "omschrijving": [],
    "risico": []
}

for _, row in val_df.iterrows():
    print(f"Processing {row['fotonummer']}")
    image_path = os.path.join(images_folder, row["fotonummer"])
    
    response = prompt(SYSTEM_PROMPT, USER_PROMPT, image_path)

    result: str = response.choices[0].message.content
    risico = result.partition("**Beoordeling van risico:**")[2].partition("**Toelichting:**")[0].strip().strip("*")
    omschrijving = result.partition("**Omschrijving van het probleem:**")[2].partition("**Omgevingsfactoren:**")[0].strip().strip("*")

    data["foto"].append(row["fotonummer"])
    data["omschrijving"].append(omschrijving)
    data["risico"].append(risico)

In [ ]:
gpt_df = pd.DataFrame(data=data)

## Run model over a bunch of images and generate a PDF with results

In [ ]:
images_folder = "../datasets/experiments/onderhoud/2025_Centrum/validatie/"

images = os.listdir(images_folder)

image_text_pairs = []

for img in images:
    print(f"Processing {img}")
    image_path = os.path.join(images_folder, img)
    
    response = prompt(SYSTEM_PROMPT, USER_PROMPT, image_path, detail="high")

    result: str = response.choices[0].message.content
    image_text_pairs.append((image_path, result))

In [ ]:
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import mm
from reportlab.pdfgen import canvas
from reportlab.platypus import Paragraph, Frame
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.utils import ImageReader
import markdown
import os

def generate_pdf(output_path, image_text_pairs, page_size=A4, margin_mm=20, image_max_height_mm=120):
    """
    Generates a PDF with each page containing a centered image and a text block below it.
    
    :param output_path: Path where the PDF will be saved.
    :param image_text_pairs: List of tuples (image_path, text), each for one page.
    :param page_size: Tuple with the page size (default: A4).
    :param margin_mm: Margin on each side in mm.
    :param image_max_height_mm: Maximum image height in mm.
    """
    c = canvas.Canvas(output_path, pagesize=page_size)
    width, height = page_size
    margin = margin_mm * mm
    image_max_height = image_max_height_mm * mm

    styles = getSampleStyleSheet()
    text_style = styles['Normal']

    for image_path, text in image_text_pairs:
        # Draw image centered horizontally
        if not os.path.exists(image_path):
            raise FileNotFoundError(f"Image not found: {image_path}")
        img = ImageReader(image_path)
        img_width, img_height = img.getSize()

        # Scale image to fit within page margins and max height
        available_width = width - 2 * margin
        scale = min(available_width / img_width, image_max_height / img_height, 1.0)
        draw_width = img_width * scale
        draw_height = img_height * scale

        x = (width - draw_width) / 2
        y = height - margin - draw_height
        c.drawImage(image_path, x, y, width=draw_width, height=draw_height)

        # Draw text block below image, allowing for newlines in the caption
        frame_height = y - margin

        # Convert newlines in text to <br/> for Paragraph formatting
        html = markdown.markdown(text.replace('\n', '<br/>'))
        paragraph = Paragraph(html, text_style)
        frame = Frame(margin, margin, width - 2 * margin, frame_height, showBoundary=0)
        frame.addFromList([paragraph], c)

        c.showPage()

    c.save()


# Example usage
# image_text_pairs = [
#     (os.path.join(images_folder, image_name), response.choices[0].message.content),
#     (os.path.join(images_folder, image_name), response.choices[0].message.content),
# ]
generate_pdf("rapport.pdf", image_text_pairs)